In [1]:
from pathlib import Path
import pandas as pd

from expense_models import load_expenses, expenses_to_dataframe

In [2]:
expenses_path = Path("2025-11-07.json")
expenses = load_expenses(expenses_path)

len(expenses)

df = expenses_to_dataframe(expenses)
df


,account_id,name,value,currency,repeat_every_unit,repeat_every
0,activo_bank,Rent,705.00,EUR,months,1
1,activo_bank,T3,86.54,EUR,months,1
2,activo_bank,Cleaning,62.50,EUR,weeks,3
3,activo_bank,Revolution food,41.00,EUR,weeks,1
4,activo_bank,Energy,35.57,EUR,months,1
5,activo_bank,Vodafone,28.50,EUR,months,1
6,activo_bank,Internet,25.75,EUR,months,1
7,activo_bank,ChatGPT,24.60,USD,months,1
8,activo_bank,Cursor,20.00,USD,months,1
9,activo_bank,Water,15.00,EUR,months,1
